In [1]:
# import libraries (non-cryptogaphic)
import random # to generate phone numbers
import pandas as pd

# import libraries (cryptographic)
import cryptography.hazmat.primitives.asymmetric.dh as dh
from cryptography.fernet import Fernet
import hashlib
import sympy
import secrets

# Generate phone numbers and store them in phone_numbers.csv file

In [2]:
random.seed(10) # to ensure same phone numbers generated every time

In [3]:
# class to generate phone numbers for grab and gojek
class PhoneNumberGenerator:
    def __call__(self, count):
        phone_numbers = random.sample(range(80000000,100000000), count)
        return phone_numbers 

class FakePhoneNumberGenerator:
    def __call__(self, count):
        numbers = random.sample(range(10000000,80000000), count) # invalid phone numbers generated
        return numbers
            
    
# class to store numbers to csv
class PhoneNumberStorageManager:
    def __init__(self):
        self.filename = "phone_numbers_padded.csv"
    def __call__(self, gojek_phone_numbers, grab_phone_numbers):
        d = {"gojek": gojek_phone_numbers, 
            "grab": grab_phone_numbers}
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()])) # create dataframe
        df.to_csv(self.filename, index = False) # store values to file "phone_numbers_padded.csv"

### Edit cell below to change phone number count and set size

In [4]:
# set set-size
set_size = 100 # both parties are to exchange sets of a pre-determined size

# set phone number count
gojek_phone_number_count = 61 # inclusive of phone numbers in common with grab
grab_phone_number_count = 91 # inclusive of phone numbers in common with gojek
common_phone_number_count = 10



In [5]:
# instantiate required classes
phone_number_generator = PhoneNumberGenerator()
fake_phone_number_generator = FakePhoneNumberGenerator()
phone_number_storage_manager = PhoneNumberStorageManager()

# generate phone numbers
phone_numbers = phone_number_generator(gojek_phone_number_count+grab_phone_number_count-common_phone_number_count)
common_phone_numbers = phone_numbers[0:common_phone_number_count]
gojek_phone_numbers = phone_numbers[0:gojek_phone_number_count]
grab_phone_numbers = common_phone_numbers + phone_numbers[gojek_phone_number_count:] 

# generate fake phone numbers
gojek_fake_phone_numbers = fake_phone_number_generator(set_size-gojek_phone_number_count)
grab_fake_phone_numbers = fake_phone_number_generator(set_size-grab_phone_number_count)

# add fake phone numbers to phone numbers
gojek_padded_set = gojek_phone_numbers + gojek_fake_phone_numbers
grab_padded_set = grab_phone_numbers + grab_fake_phone_numbers

# shuffle phone number lists
random.shuffle(gojek_padded_set)
random.shuffle(grab_padded_set)

# write phone numbers (with fake ones) to csv file
phone_number_storage_manager(gojek_padded_set, grab_padded_set)

# Define classes for the necessary for the algorithm

In [6]:
# class to generate numbers required for psi
class NumberGenerator:
    
    def generate_public_parameters(self, size):
        # method to generate p, q and factors of p-1
        p = self.generate_safe_prime(size)
        print(f"p is prime: {sympy.ntheory.isprime(p)}")
        length_of_p = len(bin(p)[2:]) # should be 1024
        print(f"Length of prime modulus, p: {length_of_p}.\nNote: Should be {size}.")
        q = (p-1)//2
        print(f"q is prime: {sympy.ntheory.isprime(q)}")
        factors_pminus1 = [1, 2, q] # since q is prime, 2q only has these 3 factors excluding itself
                                    # 2q = p-1
                                    # factors of p-1 required to compute order of generators (lagrange theorem)
        return p, factors_pminus1

    def generate_safe_prime(self, size):
        # method to generate safe prime for p
        candidate = dh.generate_parameters(2, size).parameter_numbers().p # generate 1024-bit prime number
        while True:
            # the method used from the cryptography already generates a safe prime, this portion is merely to double confirm
            is_safe_prime = sympy.ntheory.isprime((candidate-1)//2) # if safe prime, (candidate-1)/2 is prime
            if (is_safe_prime):
                break
            else:
                candidate = dh.generate_parameters(2, size).parameter_numbers().p
                print(candidate)
 
        return candidate
    
    
    def generate_random_number(self, size):
        # method to generate client's secret
        return secrets.randbits(size)
            
# class to inspect values          
class NumberInspector:
    
    def check_is_primitive_generator(self, candidate, factors_divisorminus1, divisor): # note: factors should be the factors of divisor-1
        
        # apply lagrange theorem
        for possible_order in factors_divisorminus1: # check congruence for all factors (factors is exclusive of p-1 itself)
            result = pow(candidate, possible_order, divisor) # fast modular exponentiation implemented in pow(x,y,z)
                                                             # python integers have arbitrary precisions, 
                                                             # no overflow would occur if operations done in pure python
            if (result == 1):
                return False # if candidate exponent any of the factors (1,2,q) congruent to 1modp, candidate is not a primitive
                             # generator
        return True # order of candidate == p-1, therefore candidate is a primitive generator

    
class StorageManager:
    
    def store_data(self, filename, data):
        df = pd.DataFrame(dict([(k,pd.Series(v, dtype = "str")) for k,v in data.items()])) # create dataframe
        df.to_csv(filename, index=False) # store data to file with filename


# Create psi client class

In [7]:
# client class (both grab and gojek are clients communicating directly with each other)
class Client:
    def __init__(self, name, other_party_name, private_key_size, phone_numbers, p, factors_pminus1, fernet_key):
        
        self.number_inspector = NumberInspector()
        self.number_generator = NumberGenerator()
        
        # for asymmetric encryption
        self.private_key = self.number_generator.generate_random_number(private_key_size)
        self.my_set = phone_numbers
        self.p = p # prime modulus
        self.factors_pminus1 = factors_pminus1 # to calculate order of hashed phone numbers, 
                                               # since algorithm requires them to be primitive generators
        
        # for symmetric encryption
        self.fernet_key = fernet_key
        self.f = Fernet(fernet_key)
        
        # values to track for psi algorithm
        self.my_hashed_set = None # h(x)
        self.my_self_encrypted_set = None # (h(x)^(my_secret))modp
        self.my_encrypted_set = None # (h(x)^(my_secret)(other_party_secret))modp
        self.other_party_encrypted_set = None # (h(y)^(my_secret)(other_party_secret))modp
        self.common_values = None # common phone numbers
        
        
        # create datafile for communication with another party
        
        # content to store in file
        # only need to share self_encrypted_values and other_party_encrypted_values
        # common_values to ensure both calculate the same intersection
        self.my_dict = {
            'my_self_encrypted_set': None, 
            'other_party_encrypted_set': None,
            'common_values': None
        }
        
        # filenames
        self.name = name
        self.filename = name + "_data_v1.2.csv"
        self.other_party_name = other_party_name
        self.other_party_filename = other_party_name + "_data_v1.2.csv"
        
        # create file
        self.storage_manager = StorageManager()
        self.storage_manager.store_data(self.filename, self.my_dict)

    def hash_to_primitive_root_modulo_p(self, element): 
        # method to hash phone numbers to primitive root modulo p i.e. primitive generator

        endian = "big"
        element = element.to_bytes(4, endian)
        hash_hex = hashlib.sha256(element).hexdigest() # sha3_256
        hash_int = int(hash_hex, 16)
        while True:
            # repeatedly hash until primitive root modulo p is obtained
            is_primitive_generator = self.number_inspector.check_is_primitive_generator(
                hash_int, self.factors_pminus1, self.p
            )
            if (is_primitive_generator):
                break
            else:
                hash_int = hash_int.to_bytes(32, endian)
                hash_hex = hashlib.sha256(hash_int).hexdigest()
                hash_int = int(hash_hex, 16)
                
        return hash_int
    
    def modular_exponentation(self, element):
        # compute (element^(private_key))modp
    
        return pow(element, self.private_key, self.p)
    
    def hash_set(self):
        # hash all phone numbers in my set to primitive root modulo p, one by one
        
        self.my_hashed_set = []
        
        for element in self.my_set:
            hashed_value = self.hash_to_primitive_root_modulo_p(element)
            self.my_hashed_set.append(hashed_value)
            
    def encrypt_set(self, is_other_party):
        # encrypt all elements in a given set using private_key, one by one
        
        # two scenarios to consider
        # one: encrypt set sent by the other party
        if (is_other_party):
            decrypted_other_party_set = self.receive_data("my_self_encrypted_set")
            other_party_set_int = []
            for element_string in decrypted_other_party_set:
                other_party_set_int.append(int(element_string))
            set_to_encrypt = other_party_set_int
        # two: encrypt my own set
        else:
            set_to_encrypt = self.my_hashed_set
        
        # encrypt values in given set, one by one
        encrypted_values = []
        for element in set_to_encrypt:
            encrypted_value = self.modular_exponentation(element)
            encrypted_values.append(encrypted_value)
            
        # assign the encrypted set to the correct variable
        # update csv file used for communication
        if (is_other_party):
            self.other_party_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "other_party_encrypted_set")
        else:
            self.my_self_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "my_self_encrypted_set")
                    
        
    def get_intersection(self):
        
        # get intersection
        
        my_encrypted_set = self.receive_data("other_party_encrypted_set") # read my encrypted set from the other party's file
        my_encrypted_set_int = []
        
        # convert read values to integer
        for element in my_encrypted_set:
            my_encrypted_set_int.append(int(element))
            
        # assign to correct variable
        self.my_encrypted_set = my_encrypted_set_int
        
        # get intersection
        encrypted_common_values = set(self.my_encrypted_set).intersection(self.other_party_encrypted_set)
        index_of_common_values = []
        
        # find the index of the elements in the intersection in my_encrypted_set
        for element in encrypted_common_values:
            index_of_common_values.append(self.my_encrypted_set.index(element))
            
        self.common_values = []
        
        # find the values in my own set corresponding to the index of the elements in the intersection
        for index in index_of_common_values:
            potential_phone_number = self.my_set[index]
            
            if ((potential_phone_number-79999999)>0): # remove any potential fake numbers that intersect
                self.common_values.append(potential_phone_number)
        
        # update csv file for communication
        self.send_data(self.common_values, "common_values")
    
    def encrypt_data(self, plaintext):
        # encrypt data with Fernet
        
        endian = "big"
        element = plaintext.to_bytes(128, endian) # 1024 bits == 128 bytes
        cipher_text = self.f.encrypt(element)
        return cipher_text
    
    def decrypt_data(self, ciphertext):
        # decrypt data encrypted by Fernet
        
        ciphertext_bytes = ciphertext.encode('utf-8')[2:-1] # convert from string back to bytes
        endian = "big"
        element_in_bytes = self.f.decrypt(ciphertext_bytes)
        plaintext = int.from_bytes(element_in_bytes, endian)
        return plaintext

    def send_data(self, data_to_send, column_name):
        # send data means writing to file. encrypt data with Fernet
        
        # encrypt data
        encrypted_data_to_send = []
        for element in data_to_send:
            encrypted_element = self.encrypt_data(element)
            encrypted_data_to_send.append(encrypted_element)
            
        # send data
        self.my_dict[column_name] = encrypted_data_to_send
        self.storage_manager.store_data(self.filename, self.my_dict)
        
        
    def receive_data(self, column_name):
        # receive data means reading from file (my file). decrypt data encrypted by Fernet
        
        # receive data
        encrypted_data = self.get_other_party_data()[column_name].to_list()
        
        # decrypt data
        decrypted_data = []
        for element in encrypted_data:
            if type(element) is float: # remove NaN
                continue
            decrypted_element = self.decrypt_data(element)
            decrypted_data.append(decrypted_element)
        print(f"number of elements in {self.other_party_name};{column_name}: {len(decrypted_data)}.")
        return decrypted_data
    
    def get_my_data(self):
        # read my file as dataframe (other party's file)
        
        return pd.read_csv(self.filename)
            
    def get_other_party_data(self):
        # read other party's file as dataframe
        
        return pd.read_csv(self.other_party_filename)
    
    


# Initialize context

### Edit cell below to change key size

In [8]:
# assign pre-determined variables for psi
key_size = 1024 # both private keys and large prime

In [9]:
# create key for symmetric key cryptography
fernet_key = Fernet.generate_key()

# create public parameters required for psi
number_generator = NumberGenerator()
p, factors_pminus1 = number_generator.generate_public_parameters(key_size)

# create clients
grab = Client("grab", "gojek", key_size, grab_padded_set, p, factors_pminus1, fernet_key)
gojek = Client("gojek", "grab", key_size, gojek_padded_set, p, factors_pminus1, fernet_key)



p is prime: True
Length of prime modulus, p: 1024.
Note: Should be 1024.
q is prime: True


# Get intersection

## Step 1: Hash phone numbers

In [10]:
# clients hash their own set
grab.hash_set()
gojek.hash_set()

### Clients' status after step 1
Note: hashed set is not stored in the file as it is not meant to be shared with the other party hence, files' status after step 1 is not shown

In [11]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set
0,3444554169003369415400187425444370377909224933...,6083752070806636466132091408609415889437525399...
1,7203536479102212369414139081242539081912748976...,6555446118383117843414628061923602991736146568...
2,5773315736316327673762720174221743416131441931...,8470099625806158407512470991116088653111145428...
3,5203358287300148779441149003132562463607163612...,4314342032682825508775434884617987666037471300...
4,5239384073106902936597081070089405716452234004...,4339504937591743225701797258110083974962550965...
...,...,...
95,6587301230314069546106744069227162746783023913...,6083392442958961017114113282827980873818292160...
96,3305204118985365265535072262394315086199566448...,8362019212874245816279708570292780730782352063...
97,9172421764413121270057950649496335692462033543...,3948222474783499474687611776797431125471701320...
98,9196222794520213004035745078489116002064573260...,8258531869996474717019619240419875890974984254...


## Step 2: encrypt hashed set with own private key

In [12]:
# clients self encrypt hashed set
grab.encrypt_set(False) # set is_other_party to false to encrypt own hashed set
gojek.encrypt_set(False)

### Clients' status after step 2 (value of variables in client)

In [13]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set
0,3444554169003369415400187425444370377909224933...,6083752070806636466132091408609415889437525399...,3263921956749181440677145499686084684752006165...,9674545593440033394281963112351697763774611900...
1,7203536479102212369414139081242539081912748976...,6555446118383117843414628061923602991736146568...,7457947770306558527143536955149404021165480011...,9345038646287815097320484827125812088922922656...
2,5773315736316327673762720174221743416131441931...,8470099625806158407512470991116088653111145428...,2021073630641790647119442200116469326872694212...,6052838086396054746814178842893939967680780778...
3,5203358287300148779441149003132562463607163612...,4314342032682825508775434884617987666037471300...,2588278336059197712368579441905014176454876929...,2398560332037674350539078310422029324114040441...
4,5239384073106902936597081070089405716452234004...,4339504937591743225701797258110083974962550965...,5494701683232016887666037259632397680979037439...,2148505084511986318255493859319097507994114588...
...,...,...,...,...
95,6587301230314069546106744069227162746783023913...,6083392442958961017114113282827980873818292160...,6148995899064151212569081399200917901555763453...,1400575582655891811181230428282034207155112892...
96,3305204118985365265535072262394315086199566448...,8362019212874245816279708570292780730782352063...,5488886759830578362317471167581558607736414291...,5805327444418169734495463647059934959740244389...
97,9172421764413121270057950649496335692462033543...,3948222474783499474687611776797431125471701320...,5136363453674432733200160538178083414405607578...,2859302393889894406748146127307949413121512870...
98,9196222794520213004035745078489116002064573260...,8258531869996474717019619240419875890974984254...,5216060154486152034316033144718896185173041857...,8038051445903372689072969876894252277687929601...


### Files' status after step 2 (value of variables in file - clients' variables encrypted with Fernet)

In [14]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek


gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhoxfNhtruiyouEGkznjVcZStgfwuxrm7GslzG...,NaN,NaN
1,b'gAAAAABhoxfNRZUnLyiVRcJPmHSt0aLTOjsUGbje08hV...,NaN,NaN
2,b'gAAAAABhoxfNhwuNLCtaI_hGLPRBj0HhfodeLDKwMno5...,NaN,NaN
3,b'gAAAAABhoxfNdt03WukD9HVbcW6LMqaLu_uF1JTNR8wo...,NaN,NaN
4,b'gAAAAABhoxfNeCCsUj1kvnCJhZD3s8mXWxVHJ7onSowE...,NaN,NaN
...,...,...,...
95,b'gAAAAABhoxfNGmjcpTxf2sUj5lFXvicwR3MgBwu_TQp0...,NaN,NaN
96,b'gAAAAABhoxfNR0feqILBEV6nOc72ZZd8gL4KfSzJ_fHJ...,NaN,NaN
97,b'gAAAAABhoxfNOYNvZceve_ut7PHiGlgGe2pOAtMe3YKJ...,NaN,NaN
98,b'gAAAAABhoxfNXsyuWIc07NTkXjVyBr-UAVrKAG55dQoG...,NaN,NaN


In [15]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhoxfMJlxSMwl1WRmdWaAORzzZ0tUyXaLrZln4...,NaN,NaN
1,b'gAAAAABhoxfMANidJhhAXslE_Y_CCooYVOytBDG51qp6...,NaN,NaN
2,b'gAAAAABhoxfMzHhw7f49_sQfKOH_4XMR74ZzujcFiNOa...,NaN,NaN
3,b'gAAAAABhoxfMecBOD0R12wGwhd0QKTkFxyqYeVRR6Pnc...,NaN,NaN
4,b'gAAAAABhoxfM4S-f7JoZreMLAja2MRRUBklLm_d0QjqN...,NaN,NaN
...,...,...,...
95,b'gAAAAABhoxfMWcLs18vQEEOWNqizXCVgQItgbDS8j3vD...,NaN,NaN
96,b'gAAAAABhoxfMBoNT6Ic_1dfY4E6WzE66ZyyUaPTvDDxy...,NaN,NaN
97,b'gAAAAABhoxfM0Ui6cqqSpnZ6IebOQ7U_cwUVauGgKYgx...,NaN,NaN
98,b'gAAAAABhoxfMCEB4iwVlkrUI0GR4AIcV_b4HOUTIpW2T...,NaN,NaN


## Step 3: encrypt other party's self-encrypted set with own private key


In [16]:
# clients encrypt other party's self encrypted set
grab.encrypt_set(True) # set is_other_party to true
gojek.encrypt_set(True)

number of elements in gojek;my_self_encrypted_set: 100.
number of elements in grab;my_self_encrypted_set: 100.


### Clients' status after step 3

In [17]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set
0,3444554169003369415400187425444370377909224933...,6083752070806636466132091408609415889437525399...,3263921956749181440677145499686084684752006165...,9674545593440033394281963112351697763774611900...,2791276966182981082044939233635553968278265174...,4275222955189369011712423345081545255319412359...
1,7203536479102212369414139081242539081912748976...,6555446118383117843414628061923602991736146568...,7457947770306558527143536955149404021165480011...,9345038646287815097320484827125812088922922656...,9700695397128394893822596054472428065795556355...,2298729278033937288861496486948757728603166557...
2,5773315736316327673762720174221743416131441931...,8470099625806158407512470991116088653111145428...,2021073630641790647119442200116469326872694212...,6052838086396054746814178842893939967680780778...,2519486107848062392809762788423019215835416627...,3631169528084131965605232509770350858419639711...
3,5203358287300148779441149003132562463607163612...,4314342032682825508775434884617987666037471300...,2588278336059197712368579441905014176454876929...,2398560332037674350539078310422029324114040441...,2039975788958639724081542015967646731584182772...,1758170440557639376140588206289190603837924854...
4,5239384073106902936597081070089405716452234004...,4339504937591743225701797258110083974962550965...,5494701683232016887666037259632397680979037439...,2148505084511986318255493859319097507994114588...,3855073190306603003329726608524193085531406060...,7437618935030059739414351500706244443968943626...
...,...,...,...,...,...,...
95,6587301230314069546106744069227162746783023913...,6083392442958961017114113282827980873818292160...,6148995899064151212569081399200917901555763453...,1400575582655891811181230428282034207155112892...,2957180564834338155387682562796123949941304895...,2444488342440881095626925234533573199250673431...
96,3305204118985365265535072262394315086199566448...,8362019212874245816279708570292780730782352063...,5488886759830578362317471167581558607736414291...,5805327444418169734495463647059934959740244389...,3044534109646526596487574337733940395499771742...,7120215236217864439023459758599007871757712114...
97,9172421764413121270057950649496335692462033543...,3948222474783499474687611776797431125471701320...,5136363453674432733200160538178083414405607578...,2859302393889894406748146127307949413121512870...,7109550319256780573289795883816072692832990947...,4119346127496265188759002011942851567760756730...
98,9196222794520213004035745078489116002064573260...,8258531869996474717019619240419875890974984254...,5216060154486152034316033144718896185173041857...,8038051445903372689072969876894252277687929601...,7035992349362160562625980528591625968417124982...,7372935183402472079141457713956599721380719281...


### Files' status after step 3

In [18]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek



gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhoxfNhtruiyouEGkznjVcZStgfwuxrm7GslzG...,b'gAAAAABhoxfOO8vOIqhIupiVPk95Uk2jF62ckGzwjxIx...,NaN
1,b'gAAAAABhoxfNRZUnLyiVRcJPmHSt0aLTOjsUGbje08hV...,b'gAAAAABhoxfOWq5RDHr16NJgOwwaQstzeqtqrY_WgqLK...,NaN
2,b'gAAAAABhoxfNhwuNLCtaI_hGLPRBj0HhfodeLDKwMno5...,b'gAAAAABhoxfOWnDJ_gWd-S2nFq1dB-rf-orwxNY2MDqK...,NaN
3,b'gAAAAABhoxfNdt03WukD9HVbcW6LMqaLu_uF1JTNR8wo...,b'gAAAAABhoxfO0dqgB7mNTA8drayQVFAdVutDaEVLSkFj...,NaN
4,b'gAAAAABhoxfNeCCsUj1kvnCJhZD3s8mXWxVHJ7onSowE...,b'gAAAAABhoxfOzHzDMaU6x2w3eJJR3VS8xbSWaZsF1let...,NaN
...,...,...,...
95,b'gAAAAABhoxfNGmjcpTxf2sUj5lFXvicwR3MgBwu_TQp0...,b'gAAAAABhoxfObeoaqY8O4ZaxvkZRi3xMmwlH5wtyhU_P...,NaN
96,b'gAAAAABhoxfNR0feqILBEV6nOc72ZZd8gL4KfSzJ_fHJ...,b'gAAAAABhoxfOsYmak7_sJO61x2FQ_qmk0pX-yVJZZH9b...,NaN
97,b'gAAAAABhoxfNOYNvZceve_ut7PHiGlgGe2pOAtMe3YKJ...,b'gAAAAABhoxfOyDjwZy--6CPhvh-Rmo6uzo5tlpg3V0Nb...,NaN
98,b'gAAAAABhoxfNXsyuWIc07NTkXjVyBr-UAVrKAG55dQoG...,b'gAAAAABhoxfOhxz1xDTZcRqnYxpakwb4E0sT_6i46zAs...,NaN


In [19]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhoxfMJlxSMwl1WRmdWaAORzzZ0tUyXaLrZln4...,b'gAAAAABhoxfOAMTV1s5x0KdBtoT0GJLLx4BEGJefbLnD...,NaN
1,b'gAAAAABhoxfMANidJhhAXslE_Y_CCooYVOytBDG51qp6...,b'gAAAAABhoxfOQnITE8SSTLtR6c4NZI7j1tepmdu0WlUc...,NaN
2,b'gAAAAABhoxfMzHhw7f49_sQfKOH_4XMR74ZzujcFiNOa...,b'gAAAAABhoxfO115Ecabpqct7LVsPm-YJte3WPu8wsR_7...,NaN
3,b'gAAAAABhoxfMecBOD0R12wGwhd0QKTkFxyqYeVRR6Pnc...,b'gAAAAABhoxfOqjDQH1mrBvanRH11OdIPsfaC9aObwauI...,NaN
4,b'gAAAAABhoxfM4S-f7JoZreMLAja2MRRUBklLm_d0QjqN...,b'gAAAAABhoxfOOsmoXSpFchAOs0oaTO_pJltr0H_pIwkF...,NaN
...,...,...,...
95,b'gAAAAABhoxfMWcLs18vQEEOWNqizXCVgQItgbDS8j3vD...,b'gAAAAABhoxfOr2W0eMaW9idziF9xef8HwZMgM08BWzmL...,NaN
96,b'gAAAAABhoxfMBoNT6Ic_1dfY4E6WzE66ZyyUaPTvDDxy...,b'gAAAAABhoxfOqlhPTDdalj3wZ_t6MuTzhkeDxv-xsFzJ...,NaN
97,b'gAAAAABhoxfM0Ui6cqqSpnZ6IebOQ7U_cwUVauGgKYgx...,b'gAAAAABhoxfOotYjkeRUCG8XwYiqv6z5J-B0nH2xGktE...,NaN
98,b'gAAAAABhoxfMCEB4iwVlkrUI0GR4AIcV_b4HOUTIpW2T...,b'gAAAAABhoxfO_3ChDKONZiriwgX_IPmTm5HryRGNkvWN...,NaN


## Step 4: find intersection


In [20]:
# clients find intersection
grab.get_intersection()
gojek.get_intersection()

number of elements in gojek;other_party_encrypted_set: 100.
number of elements in grab;other_party_encrypted_set: 100.


### Clients' status after step 4:

In [21]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set,
    "gojek found intersection": gojek.common_values,
    "grab found intersection:": grab.common_values}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set,gojek found intersection,grab found intersection:
0,3444554169003369415400187425444370377909224933...,6083752070806636466132091408609415889437525399...,3263921956749181440677145499686084684752006165...,9674545593440033394281963112351697763774611900...,2791276966182981082044939233635553968278265174...,4275222955189369011712423345081545255319412359...,99397525.0,99397525.0
1,7203536479102212369414139081242539081912748976...,6555446118383117843414628061923602991736146568...,7457947770306558527143536955149404021165480011...,9345038646287815097320484827125812088922922656...,9700695397128394893822596054472428065795556355...,2298729278033937288861496486948757728603166557...,96192082.0,96192082.0
2,5773315736316327673762720174221743416131441931...,8470099625806158407512470991116088653111145428...,2021073630641790647119442200116469326872694212...,6052838086396054746814178842893939967680780778...,2519486107848062392809762788423019215835416627...,3631169528084131965605232509770350858419639711...,80497694.0,80497694.0
3,5203358287300148779441149003132562463607163612...,4314342032682825508775434884617987666037471300...,2588278336059197712368579441905014176454876929...,2398560332037674350539078310422029324114040441...,2039975788958639724081542015967646731584182772...,1758170440557639376140588206289190603837924854...,95521626.0,95521626.0
4,5239384073106902936597081070089405716452234004...,4339504937591743225701797258110083974962550965...,5494701683232016887666037259632397680979037439...,2148505084511986318255493859319097507994114588...,3855073190306603003329726608524193085531406060...,7437618935030059739414351500706244443968943626...,99173089.0,99173089.0
...,...,...,...,...,...,...,...,...
95,6587301230314069546106744069227162746783023913...,6083392442958961017114113282827980873818292160...,6148995899064151212569081399200917901555763453...,1400575582655891811181230428282034207155112892...,2957180564834338155387682562796123949941304895...,2444488342440881095626925234533573199250673431...,NaN,NaN
96,3305204118985365265535072262394315086199566448...,8362019212874245816279708570292780730782352063...,5488886759830578362317471167581558607736414291...,5805327444418169734495463647059934959740244389...,3044534109646526596487574337733940395499771742...,7120215236217864439023459758599007871757712114...,NaN,NaN
97,9172421764413121270057950649496335692462033543...,3948222474783499474687611776797431125471701320...,5136363453674432733200160538178083414405607578...,2859302393889894406748146127307949413121512870...,7109550319256780573289795883816072692832990947...,4119346127496265188759002011942851567760756730...,NaN,NaN
98,9196222794520213004035745078489116002064573260...,8258531869996474717019619240419875890974984254...,5216060154486152034316033144718896185173041857...,8038051445903372689072969876894252277687929601...,7035992349362160562625980528591625968417124982...,7372935183402472079141457713956599721380719281...,NaN,NaN


Note: Last 2 columns, unlike the rest of the columns, do not have a one-to-one mapping with other values belonging to the same row i.e. values in the last 2 columns do not have any relation to the other values in the same row as it.

### Files' status after step 4

In [22]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek


gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhoxfNhtruiyouEGkznjVcZStgfwuxrm7GslzG...,b'gAAAAABhoxfOO8vOIqhIupiVPk95Uk2jF62ckGzwjxIx...,b'gAAAAABhoxfOhz10FLxYYq-PJ57x9SPMv88OC403eGxZ...
1,b'gAAAAABhoxfNRZUnLyiVRcJPmHSt0aLTOjsUGbje08hV...,b'gAAAAABhoxfOWq5RDHr16NJgOwwaQstzeqtqrY_WgqLK...,b'gAAAAABhoxfOl3DyZmLa1f1kxOM6plrCecoUao9bFUfz...
2,b'gAAAAABhoxfNhwuNLCtaI_hGLPRBj0HhfodeLDKwMno5...,b'gAAAAABhoxfOWnDJ_gWd-S2nFq1dB-rf-orwxNY2MDqK...,b'gAAAAABhoxfOU2Zb8GKz-UgiqxdjL0WLSLP_5o7iRJ6l...
3,b'gAAAAABhoxfNdt03WukD9HVbcW6LMqaLu_uF1JTNR8wo...,b'gAAAAABhoxfO0dqgB7mNTA8drayQVFAdVutDaEVLSkFj...,b'gAAAAABhoxfO_mjzO8P-1pIh4XR-zfULnFvtaTig7tPH...
4,b'gAAAAABhoxfNeCCsUj1kvnCJhZD3s8mXWxVHJ7onSowE...,b'gAAAAABhoxfOzHzDMaU6x2w3eJJR3VS8xbSWaZsF1let...,b'gAAAAABhoxfOswoE8rMxxeRrLCVyPVF6mXftQxxQRWe4...
...,...,...,...
95,b'gAAAAABhoxfNGmjcpTxf2sUj5lFXvicwR3MgBwu_TQp0...,b'gAAAAABhoxfObeoaqY8O4ZaxvkZRi3xMmwlH5wtyhU_P...,NaN
96,b'gAAAAABhoxfNR0feqILBEV6nOc72ZZd8gL4KfSzJ_fHJ...,b'gAAAAABhoxfOsYmak7_sJO61x2FQ_qmk0pX-yVJZZH9b...,NaN
97,b'gAAAAABhoxfNOYNvZceve_ut7PHiGlgGe2pOAtMe3YKJ...,b'gAAAAABhoxfOyDjwZy--6CPhvh-Rmo6uzo5tlpg3V0Nb...,NaN
98,b'gAAAAABhoxfNXsyuWIc07NTkXjVyBr-UAVrKAG55dQoG...,b'gAAAAABhoxfOhxz1xDTZcRqnYxpakwb4E0sT_6i46zAs...,NaN


In [23]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhoxfMJlxSMwl1WRmdWaAORzzZ0tUyXaLrZln4...,b'gAAAAABhoxfOAMTV1s5x0KdBtoT0GJLLx4BEGJefbLnD...,b'gAAAAABhoxfOJd_yU81QFHzc7FMSj2eNGLYqd42ocyyQ...
1,b'gAAAAABhoxfMANidJhhAXslE_Y_CCooYVOytBDG51qp6...,b'gAAAAABhoxfOQnITE8SSTLtR6c4NZI7j1tepmdu0WlUc...,b'gAAAAABhoxfOd3R8xYqk0xVHCkQ2E-DfiWzioIdl5eaS...
2,b'gAAAAABhoxfMzHhw7f49_sQfKOH_4XMR74ZzujcFiNOa...,b'gAAAAABhoxfO115Ecabpqct7LVsPm-YJte3WPu8wsR_7...,b'gAAAAABhoxfOzZa-Ybr9kBtOPw24ZXMJFAejL29lFroe...
3,b'gAAAAABhoxfMecBOD0R12wGwhd0QKTkFxyqYeVRR6Pnc...,b'gAAAAABhoxfOqjDQH1mrBvanRH11OdIPsfaC9aObwauI...,b'gAAAAABhoxfOxHT2GtPZJRfNiARPmJN_IM2UKkuUZqLU...
4,b'gAAAAABhoxfM4S-f7JoZreMLAja2MRRUBklLm_d0QjqN...,b'gAAAAABhoxfOOsmoXSpFchAOs0oaTO_pJltr0H_pIwkF...,b'gAAAAABhoxfOJlHvOtLBHiXiKgyJ8aMLZpMpo1jx6X6x...
...,...,...,...
95,b'gAAAAABhoxfMWcLs18vQEEOWNqizXCVgQItgbDS8j3vD...,b'gAAAAABhoxfOr2W0eMaW9idziF9xef8HwZMgM08BWzmL...,NaN
96,b'gAAAAABhoxfMBoNT6Ic_1dfY4E6WzE66ZyyUaPTvDDxy...,b'gAAAAABhoxfOqlhPTDdalj3wZ_t6MuTzhkeDxv-xsFzJ...,NaN
97,b'gAAAAABhoxfM0Ui6cqqSpnZ6IebOQ7U_cwUVauGgKYgx...,b'gAAAAABhoxfOotYjkeRUCG8XwYiqv6z5J-B0nH2xGktE...,NaN
98,b'gAAAAABhoxfMCEB4iwVlkrUI0GR4AIcV_b4HOUTIpW2T...,b'gAAAAABhoxfO_3ChDKONZiriwgX_IPmTm5HryRGNkvWN...,NaN


# Check results

In [24]:
# get intersection found by the two parties
gojek_found_intersection = gojek.common_values
grab_found_intersection = grab.common_values


# sort numbers for easier comparison
gojek_found_intersection.sort()
grab_found_intersection.sort()
common_phone_numbers.sort()

# summarize them in a dataframe
d = {"actual": common_phone_numbers,
    "gojek": gojek_found_intersection,
    "grab": grab_found_intersection}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,actual,gojek,grab
0,80497694,80497694,80497694
1,81093373,81093373,81093373
2,86915509,86915509,86915509
3,89312048,89312048,89312048
4,94391128,94391128,94391128
5,95521626,95521626,95521626
6,96192082,96192082,96192082
7,96485172,96485172,96485172
8,99173089,99173089,99173089
9,99397525,99397525,99397525
